In [1]:
import torch
from torch import nn
from torch.nn import Sequential
from torch.nn import Conv2d,MaxPool2d,Flatten,Linear,ReLU,Softmax

from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms


# 搭建网路

In [2]:
class Module(nn.Module):
    """
        输入图片shape：((n,3,32,32))
        n为batch_size
    """
    def __init__(self):
        super().__init__()
        self.layers = Sequential(
                Conv2d(in_channels=3,out_channels=32,kernel_size=5,
                    stride=1,padding=2,padding_mode='zeros'), # 32,32,32
                MaxPool2d(2, ceil_mode=True), # 32,16,16
                Conv2d(in_channels=32,out_channels=32,kernel_size=5,
                    stride=1,padding=2,padding_mode='zeros'), # 32,16,16,
                MaxPool2d(2, ceil_mode=True), # 32,8,8,
                Conv2d(in_channels=32,out_channels=64,kernel_size=5,
                    stride=1,padding=2,padding_mode='zeros'), # 64,8,8
                MaxPool2d(2, ceil_mode=True), # 64,4,4

                Flatten(), # 1024,
                Linear(1024,64), # 64
                ReLU(),
                Linear(64,10), # 10
                Softmax(dim=1)
        )
        
    def forward(self,x):
        return self.layers(x)
module = Module()

# 验证网络

In [3]:
data_test = torch.ones((64,3,32,32))
module(data_test).shape

torch.Size([64, 10])

# 导入数据

In [4]:
trans = transforms.ToTensor()
test_data = datasets.CIFAR10(root='./dataset/', train=False, transform=trans, download=True )
train_data = datasets.CIFAR10(root='./dataset/', train=True, transform=trans, download=True)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, drop_last=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True, drop_last=False)
n_trains = len(train_data)
n_tests = len(test_data)

Files already downloaded and verified
Files already downloaded and verified


# 训练和展示

## 参数准备

In [5]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("logs")
epochs = 10
lr = 0.001
cost = torch.nn.CrossEntropyLoss()
optimizer =  torch.optim.SGD(module.parameters(), lr)

## 训练

In [6]:
total_train_step=0
for epoch in range(epochs):
    print("-- 第 {} 轮训练 -- ".format(epoch+1))

    module.train()
    for i,data in enumerate(train_loader):
        total_train_step += 1 

        imgs, targets = data

        # 输出
        out = module(imgs)

        # loss
        loss = cost(out, targets)
        writer.add_scalar(tag="loss", scalar_value=loss, global_step=total_train_step)
        
        # 梯度清0，反向传播，梯度step
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if not total_train_step % 100:
            print("次数: {0}, loss: {1}".format(total_train_step,loss.item()))
    
        module.eval()
        total_test_loss = 0
        test_accuracy = 0

    
    with torch.no_grad():
        for i,data in enumerate(test_loader):
            imgs, targets = data
            out = module(imgs)
            total_test_loss += cost(out, targets)
            test_accuracy += (out.argmax(1)==targets).sum()
    test_accuracy = test_accuracy/n_tests
    print("-- 第 {} 轮训练精度: {}".format(epoch+1, test_accuracy))
    writer.add_scalar(tag="total_test_loss", scalar_value=loss, global_step=epoch+1)
    writer.add_scalar(tag="test_accuracy", scalar_value=test_accuracy, global_step=epoch+1)

writer.close()
    

-- 第 1 轮训练 -- 
次数: 100, loss: 2.303118944168091
次数: 200, loss: 2.3038289546966553
次数: 300, loss: 2.3031437397003174
次数: 400, loss: 2.304020643234253
次数: 500, loss: 2.301469326019287
次数: 600, loss: 2.3010239601135254
次数: 700, loss: 2.3013548851013184
-- 第 1 轮训练精度: 0.1128000020980835
-- 第 2 轮训练 -- 
次数: 800, loss: 2.302617073059082
次数: 900, loss: 2.302574396133423
次数: 1000, loss: 2.301694869995117
次数: 1100, loss: 2.3005166053771973
次数: 1200, loss: 2.30295991897583
次数: 1300, loss: 2.3030757904052734
次数: 1400, loss: 2.3021302223205566
次数: 1500, loss: 2.3043904304504395
-- 第 2 轮训练精度: 0.10970000177621841
-- 第 3 轮训练 -- 
次数: 1600, loss: 2.3020551204681396
次数: 1700, loss: 2.3035295009613037
次数: 1800, loss: 2.3017964363098145
次数: 1900, loss: 2.3038322925567627
次数: 2000, loss: 2.3012850284576416
次数: 2100, loss: 2.2992472648620605
次数: 2200, loss: 2.3027095794677734
次数: 2300, loss: 2.301649332046509
-- 第 3 轮训练精度: 0.10840000212192535
-- 第 4 轮训练 -- 
次数: 2400, loss: 2.3033244609832764
